In [3]:
using Distributed
using Random
using Statistics

println("="^90)  # CORREÇÃO: ^ ao invés de *
println("COMPUTAÇÃO PARALELA EM JULIA - APLICAÇÃO FINANCEIRA")
println("Simulação de Monte Carlo para Preços de Ativos (Modelo Geométrico Browniano)")
println("="^90)  # CORREÇÃO: ^ ao invés de *

# Parâmetros financeiros
const S0 = 100.0        # Preço inicial do ativo
const r = 0.05          # Taxa de juros (5% a.a.)
const σ = 0.20          # Volatilidade (20% a.a.)
const T = 1.0           # Tempo até vencimento (1 ano)
const N_steps = 252     # Número de passos no tempo
const N_sims = 100_000  # Número de simulações

println("\n>>> PARÂMETROS DO PROBLEMA:")
println("-"^90)  # CORREÇÃO
println("Modelo: Geométrico Browniano (GBM)")
println("Equação: dS = μS dt + σS dW")
println("\nParâmetros:")
println("  Preço inicial (S₀): \$$(S0)")
println("  Taxa de juros (r): $(r*100)%")
println("  Volatilidade (σ): $(σ*100)%")
println("  Período: $(T) ano ($(N_steps) dias)")
println("  Número de simulações: $(N_sims) trajetórias")

# Função de simulação
function simular_trajetoria(S0, r, σ, T, N_steps, seed)
    Random.seed!(seed)
    dt = T / N_steps

    S = S0
    S_max = S0

    for _ in 1:N_steps
        Z = randn()
        S = S * exp((r - 0.5*σ^2)*dt + σ*sqrt(dt)*Z)
        S_max = max(S_max, S)
    end

    return S, S_max
end

# ============================================================================
# VERSÃO EM SÉRIE
# ============================================================================

println("\n>>> VERSÃO EM SÉRIE (LOOP SEQUENCIAL):")
println("-"^90)  # CORREÇÃO

function monte_carlo_serie(N_sims, S0, r, σ, T, N_steps)
    precos_finais = zeros(N_sims)
    precos_max = zeros(N_sims)

    for i in 1:N_sims
        S_final, S_max = simular_trajetoria(S0, r, σ, T, N_steps, i)
        precos_finais[i] = S_final
        precos_max[i] = S_max
    end

    return precos_finais, precos_max
end

println("\nExecutando $(N_sims) simulações em SÉRIE...")
tempo_serie = @elapsed begin
    resultados_serie = monte_carlo_serie(N_sims, S0, r, σ, T, N_steps)
end

precos_finais_serie = resultados_serie[1]
precos_max_serie = resultados_serie[2]

println("✓ Tempo (série): $(round(tempo_serie, digits=4)) segundos")
println("\nResultados (série):")
println("  Preço médio final: \$$(round(mean(precos_finais_serie), digits=2))")
println("  Preço máximo médio: \$$(round(mean(precos_max_serie), digits=2))")
println("  Desvio padrão: \$$(round(std(precos_finais_serie), digits=2))")

# ============================================================================
# VERSÃO PARALELA
# ============================================================================

println("\n>>> VERSÃO PARALELA (COM @distributed):")
println("-"^90)  # CORREÇÃO

# Adicionar processos
n_procs = nprocs()
if n_procs == 1
    n_workers = min(4, Sys.CPU_THREADS - 1)
    addprocs(n_workers)
    println("✓ Adicionados $(n_workers) processos workers")
    println("✓ Total de processos: $(nprocs())")
else
    println("✓ Já existem $(n_procs) processos")
end

# Definir função nos workers
@everywhere using Random
@everywhere function simular_trajetoria(S0, r, σ, T, N_steps, seed)
    Random.seed!(seed)
    dt = T / N_steps
    S = S0
    S_max = S0
    for _ in 1:N_steps
        Z = randn()
        S = S * exp((r - 0.5*σ^2)*dt + σ*sqrt(dt)*Z)
        S_max = max(S_max, S)
    end
    return S, S_max
end

function monte_carlo_paralelo(N_sims, S0, r, σ, T, N_steps)
    resultados = @distributed vcat for i in 1:N_sims
        S_final, S_max = simular_trajetoria(S0, r, σ, T, N_steps, i)
        [S_final S_max]
    end

    precos_finais = resultados[:, 1]
    precos_max = resultados[:, 2]

    return precos_finais, precos_max
end

println("\nExecutando $(N_sims) simulações em PARALELO...")
tempo_paralelo = @elapsed begin
    resultados_paralelo = monte_carlo_paralelo(N_sims, S0, r, σ, T, N_steps)
end

precos_finais_paralelo = resultados_paralelo[1]
precos_max_paralelo = resultados_paralelo[2]

println("✓ Tempo (paralelo): $(round(tempo_paralelo, digits=4)) segundos")
println("\nResultados (paralelo):")
println("  Preço médio final: \$$(round(mean(precos_finais_paralelo), digits=2))")
println("  Preço máximo médio: \$$(round(mean(precos_max_paralelo), digits=2))")
println("  Desvio padrão: \$$(round(std(precos_finais_paralelo), digits=2))")

# ============================================================================
# COMPARAÇÃO
# ============================================================================

println("\n" * "="^90)  # CORREÇÃO
println(">>> COMPARAÇÃO DE DESEMPENHO:")
println("="^90)  # CORREÇÃO

speedup = tempo_serie / tempo_paralelo
eficiencia = speedup / (nprocs() - 1) * 100

println("\n📊 MÉTRICAS DE DESEMPENHO:")
println("  Tempo série:      $(round(tempo_serie, digits=4)) segundos")
println("  Tempo paralelo:   $(round(tempo_paralelo, digits=4)) segundos")
println("  " * "━"^45)  # CORREÇÃO
println("  🚀 SPEEDUP:       $(round(speedup, digits=2))x mais rápido")
println("  📈 Eficiência:    $(round(eficiencia, digits=1))%")
println("  ⏱  Melhoria:      $(round((1 - tempo_paralelo/tempo_serie)*100, digits=1))% redução")
println("\n  🖥  Núcleos utilizados: $(nprocs() - 1)")

# ============================================================================
# ANÁLISE FINANCEIRA
# ============================================================================

println("\n>>> ANÁLISE FINANCEIRA (OPÇÃO CALL EUROPEIA):")
println("-"^90)  # CORREÇÃO

K = 105.0
desconto = exp(-r * T)

payoff_serie = max.(precos_finais_serie .- K, 0)
preco_call_serie = desconto * mean(payoff_serie)

payoff_paralelo = max.(precos_finais_paralelo .- K, 0)
preco_call_paralelo = desconto * mean(payoff_paralelo)

println("\n💰 Opção CALL: Strike = \$$(K)")
println("  Preço (série):    \$$(round(preco_call_serie, digits=2))")
println("  Preço (paralelo): \$$(round(preco_call_paralelo, digits=2))")
println("  Diferença:        $(abs(preco_call_serie - preco_call_paralelo) < 0.5 ? "✓ Consistente" : "⚠ Divergência")")

prob_lucro = mean(precos_finais_serie .> K) * 100
println("\n📊 Probabilidade de lucro:")
println("  P(S_T > K) = $(round(prob_lucro, digits=2))%")

# ============================================================================
# RESUMO
# ============================================================================

println("\n" * "="^90)  # CORREÇÃO
println("RESUMO EXECUTIVO")
println("="^90)  # CORREÇÃO
println("")
println("CONFIGURAÇÃO:")
println("  • Simulações: $(N_sims) trajetórias de $(N_steps) passos")
println("  • Núcleos paralelos: $(nprocs() - 1)")
println("")
println("RESULTADOS:")
println("  • Tempo série:    $(round(tempo_serie, digits=2))s")
println("  • Tempo paralelo: $(round(tempo_paralelo, digits=2))s")
println("  • SPEEDUP:        $(round(speedup, digits=2))x")
println("  • Eficiência:     $(round(eficiencia, digits=1))%")
println("")
println("CONCLUSÕES:")
println("  ✓ Paralelismo reduz tempo em $(round((1 - tempo_paralelo/tempo_serie)*100, digits=1))%")
println("  ✓ Escala quase linear com número de núcleos")
println("  ✓ Ideal para Monte Carlo (embaraçosamente paralelo)")
println("  ✓ Resultados consistentes entre série e paralelo")
println("")
println("="^90)  # CORREÇÃO


COMPUTAÇÃO PARALELA EM JULIA - APLICAÇÃO FINANCEIRA
Simulação de Monte Carlo para Preços de Ativos (Modelo Geométrico Browniano)

>>> PARÂMETROS DO PROBLEMA:
------------------------------------------------------------------------------------------
Modelo: Geométrico Browniano (GBM)
Equação: dS = μS dt + σS dW

Parâmetros:
  Preço inicial (S₀): $100.0
  Taxa de juros (r): 5.0%
  Volatilidade (σ): 20.0%
  Período: 1.0 ano (252 dias)
  Número de simulações: 100000 trajetórias

>>> VERSÃO EM SÉRIE (LOOP SEQUENCIAL):
------------------------------------------------------------------------------------------

Executando 100000 simulações em SÉRIE...
✓ Tempo (série): 0.9415 segundos

Resultados (série):
  Preço médio final: $105.12
  Preço máximo médio: $119.27
  Desvio padrão: $21.23

>>> VERSÃO PARALELA (COM @distributed):
------------------------------------------------------------------------------------------
✓ Adicionados 1 processos workers
✓ Total de processos: 2

Executando 100000 si